# 02. Chroma 기반 RAG - 주석 포함 버전

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
CHROMA_PATH = os.getenv('CHROMA_PATH','./chroma_store')


In [3]:
!pip install -U llama-index llama-index-vector-stores-chroma

  Using cached llama_index-0.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached llama_index_indices_managed_llama_cloud-0.9.4-py3-none-any.whl.metadata (3.7 kB)
  Using cached llama_index_readers_file-0.5.4-py3-none-any.whl.metadata (5.7 kB)
  Using cached llama_index_readers_llama_parse-0.5.1-py3-none-any.whl.metadata (3.1 kB)
  Using cached beautifulsoup4-4.13.5-py3-none-any.whl.metadata (3.8 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached pypdf-6.0.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached striprtf-0.0.26-py3-none-any.whl.metadata (2.1 kB)
  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
  Using cached llama_cloud-0.1.35-py3-none-any.whl.metadata (1.2 kB)
  Using cached llama_parse-0.6.65-py3-none-any.whl.metadata (6.6 kB)
  Using cached llama_cloud_services-0.6.65-py3-none-any.whl.metadata (3.3 kB)
INFO: pip is looking at multiple versi

In [4]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, VectorStoreIndex, SimpleDirectoryReader
import chromadb

embed_model = OpenAIEmbedding(model="text-embedding-3-small")
client = chromadb.PersistentClient(path=CHROMA_PATH)
# ChromaDB의 영구 저장 클라이언트를 생성
collection = client.get_or_create_collection("workshop")
# "workshop"이라는 이름의 컬렉션을 가져오거나 없으면 새로 생성합니다
# collection은 관련된 벡터들을 그룹화하는 단위
vector_store = ChromaVectorStore(chroma_collection=collection)
# ChromaDB 컬렉션을 LlamaIndex가 사용할 수 있는 벡터 스토어로 래핑

storage_context = StorageContext.from_defaults(vector_store=vector_store)
# 위에서 만든 벡터 스토어를 사용하는 저장소 컨텍스트를 생성

docs = SimpleDirectoryReader('./data/txt').load_data()
index = VectorStoreIndex.from_documents(docs, storage_context=storage_context, embed_model=embed_model)
# 문서들을 벡터 인덱스로 변환합니다
# 각 문서가 청크로 나뉘고, 각 청크가 벡터로 변환되어 ChromaDB에 저장됩니다
# embed_model을 사용해서 텍스트를 벡터로 변환

qe = index.as_query_engine(similarity_top_k=5)
# similarity_top_k=5: 유사도가 높은 상위 5개의 청크를 검색해서 사용
print(qe.query('핵심 용어 3개를 뽑아 설명해줘'))


2025-09-14 10:49:45,677 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-14 10:49:46,886 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-14 10:49:49,108 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Document/Node: 데이터의 기본 단위로, LlamaIndex에서는 이를 활용하여 외부 데이터를 LLM과 연결하고 검색을 위한 데이터 구조를 형성합니다.

Index: 검색을 위한 데이터 구조로, LlamaIndex에서는 다양한 인덱스 종류를 제공하여 문서를 효율적으로 저장하고 검색할 수 있게 합니다.

Retriever: 관련 정보를 찾는 검색 엔진으로, LlamaIndex에서는 Retriever를 활용하여 사용자의 질문에 대한 관련성 높은 정보를 찾아줍니다.


In [4]:
# eos